# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('data/amazon_baby.sframe')

# Explore data

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [5]:
products['name'].show()

Materializing SArray

# group by products (name), count and sort by products(name) appear nums dec 

In [4]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [5]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [34]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [35]:
len(giraffe_reviews)

785

In [36]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [6]:
products['word_count'] = turicreate.text_analytics.count_words(products['review']) #word count

In [7]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 
                  'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
#use this subset of the words
#Often,ML practitioners will throw out words they consider “unimportant” before training their model. 
#This procedure can often be helpful in terms of accuracy. Here, throw out all words except for the very few above. 
#Using so few words in our model will hurt our accuracy, but help us interpret what our classifier is doing.

In [8]:
#function: takes in the word counts and returns the number of times for example ‘awesome’ appears in the reviews.
def word_counter(word, word_counting):
    if word in word_counting:
        return word_counting[word];
    else:
        return 0;

In [90]:
producst0=products[0]  #take the first item review 

In [91]:
producst0

{'name': 'Planetwise Wipe Pouch',
 'review': 'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.',
 'rating': 5.0,
 'word_count': {'recommend': 1.0,
  'moist': 1.0,
  'osocozy': 1.0,
  'keps': 1.0,
  'leak': 1.0,
  'holder': 1.0,
  'was': 1.0,
  'now': 1.0,
  'wipe': 1.0,
  'my': 2.0,
  'disappointed': 1.0,
  'wise': 1.0,
  'and': 3.0,
  'planet': 1.0,
  'it': 3.0,
  'love': 1.0,
  'early': 1.0,
  'not': 2.0,
  'highly': 1.0,
  'wipes': 1.0,
  'bags': 1.0,
  'came': 1.0,
  'does': 1.0,
  'i': 1.0},
 'awesome': 0.0,
 'great': 0.0,
 'fantastic': 0.0,
 'amazing': 0.0,
 'love': 1.0,
 'horrible': 0,
 'bad': 0.0,
 'terrible': 0,
 'awful': 0,
 'wow': 0,
 'hate': 0,
 'love1': 0,
 'positive sentiment': 1,
 'predicted_sentiment': 0.9997307390047091}

In [93]:
producst0['love1'] =  word_counter('love', producst0['word_count'])  #function can be used in this way 

In [97]:
producst0

{'name': 'Planetwise Wipe Pouch',
 'review': 'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.',
 'rating': 5.0,
 'word_count': {'recommend': 1.0,
  'moist': 1.0,
  'osocozy': 1.0,
  'keps': 1.0,
  'leak': 1.0,
  'holder': 1.0,
  'was': 1.0,
  'now': 1.0,
  'wipe': 1.0,
  'my': 2.0,
  'disappointed': 1.0,
  'wise': 1.0,
  'and': 3.0,
  'planet': 1.0,
  'it': 3.0,
  'love': 1.0,
  'early': 1.0,
  'not': 2.0,
  'highly': 1.0,
  'wipes': 1.0,
  'bags': 1.0,
  'came': 1.0,
  'does': 1.0,
  'i': 1.0},
 'awesome': 0.0,
 'great': 0.0,
 'fantastic': 0.0,
 'amazing': 0.0,
 'love': 1.0,
 'horrible': 0,
 'bad': 0.0,
 'terrible': 0,
 'awful': 0,
 'wow': 0,
 'hate': 0,
 'love1': 1.0,
 'positive sentiment': 1,
 'predicted_sentiment': 0.9997307390047091}

In [9]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda x : word_counter(word, x))
# x is in products['word_count'], Array or dict, apply this function to every row. 
#Use .apply() to build a new feature with the counts for each of the selected_words:
#we created a column ‘word_count’ with the word counts for each review. 
#create a new column in the proucts SFrame with the counts for each selected_word above, 
# .apply() can be used to create new columns in our data (our features) 
# create a column products[‘awesome’] where each row contains the number of times the word ‘awesome’ showed up 
#in the review for the corresponding product, and 0 if the review didn’t show up. 

In [10]:
products.head()

name,review,rating,word_count,awesome,great
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ...",0.0,0.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ...",0.0,0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0,'comfortable': 1.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",0.0,1.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",0.0,1.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",0.0,0.0


In [96]:
arrays=[]

for word in selected_words:
    print(word, products[word].sum()) 
    words_num=int(products[word].sum())
    arrays.append(words_num)

#.sum() method on each of the new columns you created
#Out of the selected_words, which one is most used in the dataset? Which one is least used? 

awesome 3892.0
great 55791.0
fantastic 1664.0
amazing 2628.0
love 41994.0
horrible 1110
bad 4183.0
terrible 1146
awful 687
wow 425
hate 1107


In [97]:
arrays.sort()

In [98]:
arrays

[425, 687, 1107, 1110, 1146, 1664, 2628, 3892, 4183, 41994, 55791]

# Define what is positive and negative sentiment

In [71]:
products['rating'].show()

Materializing SArray

In [16]:
#ignore all 3*  reviews, drop 3 stars data 
products = products[products['rating']!= 3]

In [17]:
#positive sentiment = 4-star or 5-star reviews, define 2,1 star as zero 
products['positive sentiment'] = products['rating'] >= 4

In [18]:
products.tail()

name,review,rating,word_count,awesome,great
Airline Seat BeltExtender - The Best ...,"Up until recently I havehated flying, putting on ...",5.0,"{'pressure': 1.0, 'so':1.0, 'relieved': 1.0, ...",0.0,0.0
Airline Seat BeltExtender - The Best ...,I bought this as afather's day gift for my ...,5.0,"{'highly': 1.0, 'again':1.0, 'thanked': 1.0, ...",0.0,0.0
Squeasy Snacker 6ozSilicone Reusable Food ...,"I love that these canhold purees OR liquids, ...",5.0,"{'cool': 1.0, 'oz': 1.0,'sizes': 1.0, 'more': ...",0.0,1.0
Squeasy Snacker 6ozSilicone Reusable Food ...,"I love this product, itmakes my life easier. ...",5.0,"{'forward': 1.0, 'look':1.0, 'works': 1.0, ...",0.0,1.0
Squeasy Snacker 6ozSilicone Reusable Food ...,This reusable squeezebottle is the best I ...,5.0,"{'product': 1.0,'definitely': 1.0, ...",1.0,0.0
Baby Teething Necklacefor Mom Pretty Donut ...,Such a great idea! veryhandy to have and look ...,5.0,"{'teething': 1.0, 'of':1.0, 'such': 1.0, ...",0.0,2.0
Baby Teething Necklacefor Mom Pretty Donut ...,This product rocks! Itis a great blend of ...,5.0,"{'latex': 1.0, 'free':2.0, 'pvc': 1.0, ...",0.0,1.0
Abstract 2 PK Baby /Toddler Training Cup ...,This item looks great andcool for my kids....I ...,5.0,"{'its': 1.0, 'i': 1.0,'kids': 1.0, 'in': 1.0, ...",0.0,2.0
"Baby Food Freezer Tray -Bacteria Resistant, BPA ...",I am extremely happy withthis product. I have ...,5.0,"{'way': 1.0, 'an': 1.0,'looking': 1.0, 'pare ...",0.0,0.0
Best 2 Pack Baby CarShade for Kids - Window ...,I love this product verymush . I have bought ...,5.0,"{'of': 1.0, 'also': 1.0,'well': 1.0, 'mush': ...",0.0,0.0


In [19]:
products['positive sentiment'].show()

Materializing SArray

# Train our sentiment classifier 
word counts for all words as features for our sentiment classifier.

In [22]:
train_data,test_data = products.random_split(.8,seed=0) #split train, test data 

In [40]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='positive sentiment', 
                                                        features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 3.662173     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 4.215052     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 5.015405     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 7.501030     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Evaluate the sentimental model

In [78]:
sentiment_model.evaluate(test_data,metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0    |        1.0         |        1.0         | 27976 | 5328 |
 |   1e-05   | 0.847972972972973  | 0.9975693451529882 | 27976 | 5328 |
 |   2e-05   | 0.829954954954955  | 0.9971761509865599 | 27976 | 5328 |
 |   3e-05   | 0.818506006006006  | 0.9969616814412353 | 27976 | 5328 |
 |   4e-05   | 0.8109984984984985 | 0.9967472118959108 | 27976 | 5328 |
 |   5e-05   | 0.8057432432432432 | 0.9966042321990277 | 27976 | 5328 |
 |   6e-05   | 0.7991741741741741 | 0.9962825278810409 | 27976 | 5328 |
 |   7e-05   | 0.7952327327327328 | 0.9961752931083786 | 27976 | 5328 |
 |   8e-05   | 0.7920420420420421 | 0.9961038032599371 | 27976 | 5328 |
 |   9e-05   | 0.7882882882882

# Apply the sentiment classifier to better understand the Giraffe reviews

In [79]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')
#create a new column predicted sentiment (sentiment_model) showing with probability

In [80]:
products

name,review,rating,word_count,awesome,great
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'moist': 1.0, 'osocozy': ...",0.0,0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0,'comfortable': 1.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'little': 1.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",0.0,1.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",0.0,1.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0,'recommend': 1.0, ...",0.0,0.0


In [81]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether'] #one product 

In [82]:
giraffe_reviews

name,review,rating,word_count,awesome,great
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'purchase': 1.0,'teething': 1.0, ...",0.0,1.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'a': 1.0, 'is': 1.0,'when': 1.0, 'him': 1.0, ...",0.0,1.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'made': 1.0, 'of': 1.0,'packaging': 1.0, 'no': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'another': 1.0, 'out':1.0, 'run': 1.0, 'lost': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'disappointed': 1.0,'will': 1.0, 'take': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'late': 1.0, 'perfect':1.0, 'pack': 1.0, 'on ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'delight': 1.0, 'in':1.0, 'other': 1.0, ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'inside': 1.0, 'water':1.0, 'don': 1.0, 'up': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'its': 1.0, 'fine': 1.0,'is': 1.0, 'which': 1.0, ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5.0,"{'belongs': 1.0, 'dog':1.0, 'if': 1.0, 'bewa ...",0.0,0.0


# Sort the Giraffe reviews according to predicted sentiment

In [83]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [84]:
giraffe_reviews

name,review,rating,word_count,awesome,great,fantastic
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'use': 1.0, 'get': 1.0,'will': 1.0, 'where': ...",0.0,2.0,0.0
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'explore': 1.0,'adventures': 1.0, ...",0.0,1.0,0.0
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'09': 1.0, '11': 1.0,'recommended': 1.0, ' ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'parents': 1.0,'recommend': 1.0, ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'future': 1.0, 'use':1.0, 'am': 1.0, ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'penny': 1.0, 'worth':1.0, 'definitely': 1.0, ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'teether': 1.0, 'want':1.0, 'price': 1.0, ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5.0,"{'must': 1.0, 'nuts':1.0, 'we': 1.0, 'thei ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'well': 1.0, 'natural':1.0, 'makers': 1.0, ...",0.0,0.0,0.0
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4.0,"{'dye': 1.0, 'of': 1.0,'cause': 1.0, 'fade': ...",0.0,1.0,0.0


In [85]:
giraffe_reviews.tail()

name,review,rating,word_count,awesome,great
Vulli Sophie the GiraffeTeether ...,My 7 1/2 month oldgrandbaby was sittng up ...,1.0,"{'sopie': 1.0, 'with':1.0, 'be': 1.0, 'shou ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,I was so looking forwardto getting this for my ...,1.0,"{'vulli': 1.0, 'by': 1.0,'two': 1.0, 'garanima ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,Received the product andsmells like cheap rub ...,1.0,"{'was': 1.0, 'as': 1.0,'still': 1.0, 'but': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,I got one of these as ashowe gift that my baby ...,1.0,"{'ask': 1.0, 'off': 1.0,'rubbed': 1.0, 'toy': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2.0,"{'mom': 1.0, 'teethers':1.0, 'btw': 1.0, 'oth ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,When I received this thepaint was peeling off in ...,1.0,"{'worth': 1.0, 'for':1.0, 'about': 1.0, ...",0.0,1.0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1.0,"{'they': 1.0,'naturally': 1.0, ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1.0,"{'exaggerated': 1.0,'are': 1.0, 'down': 1.0, ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1.0,"{'ready': 1.0, 'work':1.0, 'too': 1.0, 'just': ...",0.0,0.0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1.0,"{'amazon': 1.0, 'from':1.0, 'one': 1.0, 'by': ...",0.0,0.0


## Show the most positive reviews

In [86]:
giraffe_reviews[0]['review']

"As a mother of 16month old twins; I bought Sophie [1 for each, of course] when they were 4 months old after careful reading of all reviews. I heard great things about Sophie and wanted to give her a try. At 4 months babies can't do much more than grasp and semi gnaw on Sophie. For many months I had to squeeze Sophie myself [which I personally enjoyed] and set it on their laps. They LOVED Sophie. The squeak is LOUD and sounds exactly like a dog's squeaky chew toy, just for those who are wondering.As they grew and their motor skills developed to each milestone, Sophie gained more and more individual babytime. The twins were able to squeeze her themselves and chew on her around the clock. They love to throw her, stretch her, squeeze her, chew on her, drool on her... you name it, they have done it. One of the two Sophie's took an extended vacation out in the back yard [unbeknownst to me] and once found, a little water had her looking like a champ again... ready to face another day of play

In [87]:
giraffe_reviews[1]['review']

'I\'ll be honest...I bought this toy because all the hip parents seem to have one too and I wanted to be a part of the "hip parent" crowd. The price-tag was somewhat of a deterent but I prevailed and purchased this teether for my daughter.At first, Lily didn\'t know what to make of of Sophie and showed little interest in the polka-dotted creature. I continued to introduce Lily to Sophie and kept the toy in the carrier so that it was on-hand during transitions. Eventually, Lily discovered what a wonderful experience it was to gnaw on the hooves and ears and these two have never been far apart since.Lily really enjoys gumming all the different parts of Sophie like no other teether we have. The size of the toy is great as it is somewhat substantial and so easy for a little one to grasp and hold onto. Lily really enjoys hearing Sophie squeak and will smile whenever Sophie makes a noise or pops her head up from Mommy\'s lap to say hello.People have stopped and commented on Sophie and to the

# Most negative reivews

In [88]:
giraffe_reviews[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [89]:
giraffe_reviews[-2]['review']

'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 

# Create a sentiment analysis model using only the selected_words as features

In [23]:
selected_words_model = turicreate.logistic_classifier.create(train_data,
                                                     target='positive sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.752672     | 0.847708          | 0.846385            |

| 5         | 6        | 1.908018     | 0.847708          | 0.846385            |

| 6         | 7        | 2.063725     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [29]:
coef = selected_words_model.coefficients.sort('value', ascending=False)
#Call new model: selected_words_model.
#examine the weights the learned classifier assigned to each of the 11 words in selected_words 
#and gain intuition as to what the ML algorithm did for your data using these features.
#In Turi Create, a learned model, such as the selected_words_model, has a field 'coefficients',
#which lets you look at the learned coefficients.

In [101]:
coef.print_rows(num_rows=12) 
#printing the list of coefficients, use print_rows(rows=12) to print ALL coefficients.

+-------------+-------+-------+----------------------+----------------------+
|     name    | index | class |        value         |        stderr        |
+-------------+-------+-------+----------------------+----------------------+
|     love    |  None |   1   |  1.359268866922504   | 0.02806830015209953  |
| (intercept) |  None |   1   |  1.3365913848877726  | 0.008929969787655916 |
|   awesome   |  None |   1   |  1.133534666034134   | 0.08399643983187562  |
|   amazing   |  None |   1   |  1.100093311366018   |  0.0995477626046598  |
|  fantastic  |  None |   1   |   0.88580475688142   | 0.11167591293399651  |
|    great    |  None |   1   |  0.8630655001196439  | 0.018955052444376685 |
|     wow     |  None |   1   | -0.00953823606768843 | 0.16046411224711649  |
|     bad     |  None |   1   | -0.9914778800650664  | 0.03848428664699063  |
|     hate    |  None |   1   | -1.3484407222463202  | 0.07715698604297323  |
|    awful    |  None |   1   | -2.0529082040313598  | 0.1009973

In [32]:
coef.tail()

name,index,class,value,stderr
awesome,None,1,1.133534666034134,0.08399643983187562
amazing,None,1,1.100093311366018,0.0995477626046598
fantastic,None,1,0.88580475688142,0.11167591293399651
great,None,1,0.8630655001196439,0.018955052444376685
wow,None,1,-0.00953823606768843,0.16046411224711649
bad,None,1,-0.9914778800650664,0.03848428664699063
hate,None,1,-1.3484407222463202,0.07715698604297323
awful,None,1,-2.0529082040313598,0.10099735435259247
terrible,None,1,-2.2236614360851346,0.07731736203785744
horrible,None,1,-2.251335236759102,0.08020249388788425


In [33]:
selected_words_model.summary

<bound method Model.summary of Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 12
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 11
Number of unpacked features    : 11

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 2.0885

Settings
--------
Log-likelihood                 : 52926.808

Highest Positive Coefficients
-----------------------------
love                           : 1.3593
(intercept)                    : 1.3366
awesome                        : 1.1335
amazing                        : 1.1001
fantastic                      : 0.8858

Lowest Negative Coefficients
----------------------------
horrible     

In [102]:
selected_words_model.evaluate(test_data) #model 2

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 |      1       |        0        |  159  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670876317,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 53

In [103]:
sentiment_model.evaluate(test_data) #model 1

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 |      1       |        0        |  1344 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187232191,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0

In [34]:
selected_words_model.evaluate(test_data, metric='roc_curve')
#What is the accuracy of the selected_words_model on the test_data?
#What was the accuracy of the sentiment_model that we learned using all the word counts 
#What is the accuracy majority class classifier on this task? 
#How do you compare the different learned models with the baseline approach 
#where we are just predicting the majority class? 
#the majority class classifier, which simply predicts that every data point is from the most common class. 
#This is baseline is something we definitely want to beat with models we learn from data.

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328 |
 |   1e-05   |        1.0         | 1.0 | 27976 | 5328 |
 |   2e-05   |        1.0         | 1.0 | 27976 | 5328 |
 |   3e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 |   4e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 |   5e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 |   6e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 |   7e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 |   8e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 |   9e-05   | 0.9998123123123123 | 1.0 | 27976 | 5328 |
 +-----------+--------------------+-----+-------+------+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_row

# investigate a product named ‘Baby Trend Diaper Champ’. 
Just like we did for the reviews for the giraffe toy,
select all reviews where the product name is ‘Baby Trend Diaper Champ’. 

In [35]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [36]:
len(diaper_champ_reviews)

298

In [38]:
diaper_champ_reviews['rating'].show()

Materializing SArray

In [67]:
diaper_champ_reviews = diaper_champ_reviews.sort('rating', ascending=False)
#sort by ratings 

In [68]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')
# all words model , 1st model

In [70]:
diaper_champ_reviews.head() 
#What is the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’  

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I read many reviews aboutdiaper disposals and ...,5.0,"{'fine': 1.0, 'up': 1.0,'stick': 1.0, 'genie': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'parent': 1.0, 'makes': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0, 'worth':1.0, 'other': 2.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'bulk':1.0, 'any': 1.0, 't': ...",0.0,1.0,0.0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'be': 1.0, 'won': 1.0,'open': 1.0, 'if': 1.0, ...",0.0,1.0,0.0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'helps': 1.0, 'slower':1.0, 'changings': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'looking':1.0, 'product': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'seems':1.0, 'all': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'sell': 1.0, 'not': 2.0, ...",0.0,0.0,0.0


In [71]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [72]:
diaper_champ_reviews.head()
#What is the most ‘predicted_sentiment’ for ‘Baby Trend Diaper Champ’  

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'parent': 1.0, 'makes': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0, 'worth':1.0, 'other': 2.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'bulk':1.0, 'any': 1.0, 't': ...",0.0,1.0,0.0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'be': 1.0, 'won': 1.0,'open': 1.0, 'if': 1.0, ...",0.0,1.0,0.0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'helps': 1.0, 'slower':1.0, 'changings': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'looking':1.0, 'product': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'seems':1.0, 'all': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'sell': 1.0, 'not': 2.0, ...",0.0,0.0,0.0


In [55]:
diaper_champ_reviews['predicted_sentiment_selected_words'] = selected_words_model.predict(diaper_champ_reviews, 
                                                                           output_type='probability')
#2nd model 

In [56]:
diaper_champ_reviews.head()

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I am the mother of 9children and this is the ...,5.0,"{'saves': 1.0, 'which':1.0, 'custom': 1.0, ' ...",0.0,0.0,0.0
Baby Trend Diaper Champ,this works really well.I found it easier than ...,5.0,"{'no': 1.0, 'few': 1.0,'nail': 1.0, 'broken': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,my son is now 3 monthsold and when he was born ...,5.0,"{'investment': 1.0,'handle': 1.0, 'baby': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1.0, 'that':1.0, 'handle': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,i love this diaperchamp..not only does it ...,5.0,"{'recommend': 1.0,'totally': 1.0, 'child': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,You really can notappreciate this device ...,5.0,"{'reviews': 1.0,'negative': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,You cannot imagine howmuch money you'll save ...,5.0,"{'wonderfull': 1.0,'bag': 1.0, 'change': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'for': 1.0, 'settle':1.0, 'than': 1.0, 'fi ...",0.0,0.0,0.0
Baby Trend Diaper Champ,Worth every single pennyand then some! I ...,5.0,"{'market': 1.0, 'on':1.0, 'best': 1.0, 'we ...",0.0,0.0,0.0
Baby Trend Diaper Champ,...make sure I tellpeople about this ...,5.0,"{'design': 1.0, 'good':1.0, 'great': 1.0, ...",0.0,1.0,0.0


In [57]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.7919288370624482]

In [58]:
diaper_champ_reviews[(diaper_champ_reviews['review'].contains('Baby Luke'))]

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	float
	terrible	int
	awful	int
	wow	int
	hate	int
	positive sentiment	int
	predicted_sentiment	float
	predicted_sentiment_selected_words	float

Rows: Unknown

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'around': 1.0, 'bulk': 1.... |   0.0   |  1.0  |    0.0    |   0.0   | 0.0  |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+--------------------+
| horrible | bad | terrible | awful | wow | hate | positive sentiment |
+----------+-----+----------+-------+-----+------+--------------------+
|    0     | 0.0 |    0     |   0   |  0  |  0   |         1          |
+----------+-----+----------+-------+-----+------+--------------------+
+---------------------+-------------------------------+
| predicted_sentiment | predicted_sentiment_select... |
+---------------------+-------------------------------+
|  0.9999999999302822 |       0.900218694809364       |
+---------------------+-------------------------------+
[? rows x 18 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [104]:
diaper_champ_reviews[0]['review']

"I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out."

In [106]:
diaper_champ_reviews[0]['word_count']

{'key': 1.0,
 'have': 1.0,
 'pieces': 1.0,
 'between': 1.0,
 'cushioned': 1.0,
 'piece': 1.0,
 'take': 1.0,
 'overloading': 1.0,
 'be': 1.0,
 'looking': 1.0,
 're': 1.0,
 'stinkiness': 1.0,
 'retain': 1.0,
 'now': 1.0,
 'stink': 1.0,
 'once': 1.0,
 'of': 2.0,
 'below': 1.0,
 'rest': 1.0,
 'so': 1.0,
 'head': 1.0,
 'never': 1.0,
 'foam': 3.0,
 'careful': 1.0,
 'the': 5.0,
 'diaper': 1.0,
 'snagged': 1.0,
 'then': 1.0,
 'figured': 1.0,
 'week': 1.0,
 'can': 2.0,
 'change': 1.0,
 'smells': 1.0,
 'no': 2.0,
 'wonderful': 1.0,
 'mistake': 1.0,
 'pulled': 1.0,
 'big': 1.0,
 'worked': 1.0,
 '16': 1.0,
 'review': 1.0,
 'has': 1.0,
 'and': 3.0,
 'months': 1.0,
 'in': 1.0,
 've': 1.0,
 'that': 4.0,
 'part': 1.0,
 'had': 1.0,
 'what': 1.0,
 'a': 4.0,
 'we': 3.0,
 'material': 1.0,
 'is': 2.0,
 'explain': 1.0,
 'exactly': 1.0,
 'this': 1.0,
 'it': 5.0,
 'for': 3.0,
 'to': 2.0,
 'loner': 1.0,
 'clean': 1.0,
 'keeping': 1.0,
 'replacement': 1.0,
 'out': 5.0,
 'experienced': 1.0,
 'easy': 1.0,
 'read'